In [22]:
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('~')
from analysts_tools.growth import *
    #Procurement tools
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
import datetime

In [40]:
sells_date_inicio = (datetime.datetime.today() - datetime.timedelta(days=4)).strftime('%Y-%m-%d') #1 months
sells_date_fin = (datetime.datetime.today() - datetime.timedelta(days=3)).strftime('%Y-%m-%d') #1 months
print(sells_date_inicio,sells_date_fin)

2023-09-14 2023-09-15


In [41]:
df_margin = get_fresh_query_result("https://internal-redash.federate.frubana.com/",165821,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [42]:
#file1 = pd.read_csv('SH_GRADS2023-09-14SPOGROUPS_9a16.csv')
file2 = pd.read_csv('SH_GRADS2023-09-14BOGGROUPS_9a16.csv')
file3 = pd.read_csv('SH_GRADS2023-09-14MDEGROUPS_9a16.csv')
#file4 = pd.read_csv('SH_GRADS2023-09-14CMXGROUPS_9a16.csv')
file5 = pd.read_csv('SH_GRADS2023-09-14VCPGROUPS_9a16.csv')
file6 = pd.read_csv('SH_GRADS2023-09-14BHZGROUPS_9a16.csv')
#file7 = pd.read_csv('SH_GRADS2023-09-14CWBGROUPS_9a16.csv')
#file8 = pd.read_csv('SH_GRADS2023-09-14BAQGROUPS_9a16.csv')

df_groups = pd.concat([#file1, 
                       file2, 
                       file3, 
                       #file4, 
                       file5, 
                       file6#, 
                       #file7,
                       #file8
                       ], ignore_index=True) #, 

In [43]:
df_groups.groupby(['city','group']).count()

customer_id  todays_date
city group                            
BHZ  control         1260         1260
     test            1261         1261
BOG  control         4272         4272
     test            4273         4273
MDE  control         1309         1309
     test            1309         1309
VCP  control          847          847
     test             847          847

In [44]:
df_groups_info = df_groups.groupby(['group']).size().reset_index()
df_groups_info = df_groups_info.rename(columns={0: 'total_users'})

In [45]:
df_groups_info

,group,total_users
0,control,7688
1,test,7690


In [46]:
query = """
SELECT 
    s.identifier_value AS region_code,
    DATE(fs.order_submitted_date) AS submit_date,
    --fs.order_close_date AS close_date,
    --fs.order_id,
    dc.source_id AS customer_id,
    --cat.parent_description as category,
    --cat.description AS subcategory,
    dp.card_id,
    --dp.source_id AS sku_id,
    dp.card_description AS product_name,
    --fs.product_quantity_x_step_unit AS cant,
    CASE
        WHEN s.identifier_value IN ('BOG','BAQ','MDE') THEN fs.gmv_pxq_local/3776
        WHEN s.identifier_value IN ('SPO','BHZ','CWB','VCP') THEN fs.gmv_pxq_local/4.75
        WHEN s.identifier_value IN ('CMX') THEN fs.gmv_pxq_local/19.65
    ELSE fs.gmv_pxq_local END AS gmv_usd,
    CASE
        WHEN s.identifier_value IN ('BOG','BAQ','MDE') THEN fsd.product_discount/3776
        WHEN s.identifier_value IN ('SPO','BHZ','CWB','VCP') THEN fsd.product_discount/4.75
        WHEN s.identifier_value IN ('CMX') THEN fsd.product_discount/19.65
    ELSE fsd.product_discount END AS discount_applied,
    CASE
        WHEN (fsd.adjustment_reason ILIKE 'SH\_GRAD%' OR fsd.adjustment_reason ILIKE 'EXTRA\_INV\_GRAD%') THEN 'EXP'
        WHEN fsd.adjustment_reason IS NULL THEN 'NO DISCOUNT'
    ELSE 'OTHERS DCT' END AS responsable,
    fsd.adjustment_reason--,
    --bo.source_id AS offer_id
    
FROM dpr_sales.fact_sales                   fs
INNER JOIN dpr_shared.dim_customer          dc  ON dc.customer_id = fs.dim_customer
INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
LEFT JOIN dpr_sales.fact_sales_discounts    fsd ON fs.order_item_id = fsd.order_item_id 
LEFT JOIN dpr_sales.dim_offer               bo  ON bo.offer_id = fsd.dim_offer     

WHERE 
    fs.gmv_enabled = TRUE
    AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
    AND fs.fb_order_status_id  IN (1,6,7,8)
    AND fs.is_deleted = FALSE
    AND fs.dim_status = 1
    AND dp.is_slot = 'false'
    AND DATE(fs.order_submitted_date) between '{Fecha_start}' AND '{Fecha_end}'
    --AND (fsd.adjustment_reason ILIKE 'SH\_GRAD%' OR fsd.adjustment_reason ILIKE 'SH\_FARM%')
    AND dc.source_id IN {clientes}
""".format(Fecha_start = sells_date_inicio, Fecha_end = sells_date_fin, clientes = tuple(set(df_groups.customer_id.unique())))

df = run_read_dwd_query(query)
df[["gmv_usd", "discount_applied"]] = df[["gmv_usd", "discount_applied"]].astype('float32')

df['discount_applied'].fillna(0, inplace=True)

df = pd.merge(df, df_groups[['customer_id','group']], left_on='customer_id', right_on='customer_id', how='inner')
df = pd.merge(df, df_margin[['margin','card_id']], left_on='card_id', right_on='card_id', how='inner')

df["cash_margin"] = df.gmv_usd * df.margin
df["net_cash_margin"] = df.cash_margin - df.discount_applied

In [47]:
df[['customer_id','group']].drop_duplicates().group.value_counts()

test       2199
control    2130
Name: group, dtype: int64

In [48]:
df2 = df#.loc[df.region_code == 'VCP'].copy()

In [49]:
# Pivot table for GMV
pivot_gmv = pd.pivot_table(df2.loc[df2.group == 'test'], values='gmv_usd', index='customer_id', columns='responsable', aggfunc='sum', margins=True, margins_name='Total')

# Pivot table for discount
pivot_discount = pd.pivot_table(df2.loc[df2.group == 'test'], values='discount_applied', index='customer_id', columns='responsable', aggfunc='sum', margins=True, margins_name='Total')

# Pivot table for cash_margin
pivot_cash = pd.pivot_table(df2.loc[df2.group == 'test'], values='cash_margin', index='customer_id', columns='responsable', aggfunc='sum', margins=True, margins_name='Total')

# Pivot table for net_cash_margin
pivot_net_cash = pd.pivot_table(df2.loc[df2.group == 'test'], values='net_cash_margin', index='customer_id', columns='responsable', aggfunc='sum', margins=True, margins_name='Total')

################################
################################
################################

# Pivot table for GMV
pivot_gmv_control = pd.pivot_table(df2.loc[df2.group == 'control'], values='gmv_usd', index='customer_id', columns='responsable', aggfunc='sum', margins=True, margins_name='Total')

# Pivot table for discount
pivot_discount_control = pd.pivot_table(df2.loc[df2.group == 'control'], values='discount_applied', index='customer_id', columns='responsable', aggfunc='sum', margins=True, margins_name='Total')

# Pivot table for cash_margin
pivot_cash_control = pd.pivot_table(df2.loc[df2.group == 'control'], values='cash_margin', index='customer_id', columns='responsable', aggfunc='sum', margins=True, margins_name='Total')

# Pivot table for net_cash_margin
pivot_net_cash_control = pd.pivot_table(df2.loc[df2.group == 'control'], values='net_cash_margin', index='customer_id', columns='responsable', aggfunc='sum', margins=True, margins_name='Total')

In [50]:
# Concatenate the pivot tables side by side
comparison_table1 = pd.concat([pivot_cash, pivot_net_cash], axis=1)

# Display the comparison table
comparison_table1

responsable,EXP,NO DISCOUNT,OTHERS DCT,Total,EXP,NO DISCOUNT,OTHERS DCT,Total
customer_id,,,,,,,,
-254818,NaN,NaN,3.589625,3.589625,NaN,NaN,-1.557843,-1.557843
-254622,NaN,2.001990,7.667767,9.669757,NaN,2.001990,5.569885,7.571875
-253612,NaN,NaN,2.755615,2.755615,NaN,NaN,-0.040656,-0.040656
-253420,NaN,3.286150,16.523956,19.810105,NaN,3.286150,4.501472,7.787621
-252635,NaN,3.499920,1.825672,5.325593,NaN,3.499920,-0.438767,3.061153
...,...,...,...,...,...,...,...,...
355242268,NaN,NaN,1.290894,1.290894,NaN,NaN,-0.348015,-0.348015
355260261,1.368686,NaN,NaN,1.368686,0.156055,NaN,NaN,0.156055
355262398,NaN,0.312740,0.110046,0.422786,NaN,0.312740,-0.127626,0.185114


In [51]:
# Concatenate the pivot tables side by side
comparison_table2 = pd.concat([pivot_cash_control, pivot_net_cash_control], axis=1)

# Display the comparison table
comparison_table2

responsable,NO DISCOUNT,OTHERS DCT,Total,NO DISCOUNT,OTHERS DCT,Total
customer_id,,,,,,
-254982,1.025303,1.921671,2.946974,1.025303,-2.568879,-1.543576
-254755,NaN,1.864824,1.864824,NaN,-0.150075,-0.150075
-254074,15.013199,4.043460,19.056658,15.013199,1.875287,16.888486
-252754,5.844236,14.937815,20.782050,5.844236,3.771925,9.616161
-252682,8.189572,14.334959,22.524531,8.189572,11.591621,19.781193
...,...,...,...,...,...,...
355261957,0.315276,1.981292,2.296568,0.315276,0.665503,0.980779
355265809,NaN,6.059206,6.059206,NaN,3.153944,3.153944
355284746,0.713649,0.643553,1.357202,0.713649,0.203802,0.917451


In [52]:
df3 = comparison_table1.loc['Total'].to_frame().T

df4 = comparison_table2.loc['Total'].to_frame().T

In [53]:
df3

responsable,EXP,NO DISCOUNT,OTHERS DCT,Total,EXP,NO DISCOUNT,OTHERS DCT,Total
Total,2308.992937,5163.491728,10356.190004,17828.674669,486.900138,5163.491728,2847.771408,8498.163273


In [54]:
df4

responsable,NO DISCOUNT,OTHERS DCT,Total,NO DISCOUNT,OTHERS DCT,Total
Total,5673.290788,10658.628642,16331.919431,5673.290788,3178.963801,8852.254589
